In [0]:
booking_data = spark.sql("select * from zoom_cars_data.default.bookings_data")

In [0]:
booking_data.show()

In [0]:
customer_data = spark.sql("select * from zoom_cars_data.default.customers_data")

In [0]:
customer_data.show()

In [0]:
# Create an empty DataFrame with the defined schema
target_table = customer_data.join(booking_data, ["customer_id"], "leftanti").select(customer_data["*"])

# Show the empty table
target_table.show()


In [0]:
new_data_df = customer_data



# Merge the both table base

In [0]:
%python
from pyspark.sql.functions import col
update_df = new_data_df.alias("new").join(
    target_table.alias("target"), 
    ["booking_id", "customer_id"], 
    "inner"
).where(
    col("new.status") != "cancelled"
).select("new.*")

insert_df = new_data_df.alias("new").join(
    target_table.alias("target"), 
    ["booking_id", "customer_id"], 
    "leftanti"
).select("new.*")

cancelled_df = new_data_df.filter(
    col("status") == "cancelled"
).select("booking_id", "customer_id")

merged_df = target_table.alias("target").join(
    update_df.alias("update"), 
    ["booking_id", "customer_id"], 
    "leftanti"
).union(update_df)

merged_df = merged_df.union(insert_df)

merged_df = merged_df.join(
    cancelled_df.alias("cancelled"), 
    ["booking_id", "customer_id"], 
    "leftanti"
)

display(merged_df)